In [ ]:
import pygrib
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor


In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from PIL import Image

def add_image_with_title_to_presentation(image_path, presentation, text):
    # Usar un diseño de diapositiva en blanco con título horizontal en la parte superior
    slide_layout = presentation.slide_layouts[6]
    slide = presentation.slides.add_slide(slide_layout)

    # Obtener las dimensiones de la imagen
    with Image.open(image_path) as img:
        img_width, img_height = img.size

    # Dimensiones máximas (manteniendo la relación de aspecto)
    max_width = presentation.slide_width - Inches(0)  # dejar algo de margen
    max_height = presentation.slide_height - Inches(0)  # dejar espacio para el título

    # Calcular el factor de escala
    scale_factor = min(max_width / img_width, max_height / img_height)

    # Calcular las nuevas dimensiones
    new_width = img_width * scale_factor
    new_height = img_height * scale_factor

    # Calcular la posición centrada
    img_left = (presentation.slide_width - new_width) / 2
    img_top = (presentation.slide_height - new_height) / 2

    # Agregar imagen
    slide.shapes.add_picture(image_path, img_left, img_top, width=new_width, height=new_height)

    # Agregar cuadro de texto en la parte superior
    left = Inches(-3)
    top = Inches(0.2)
    width = presentation.slide_width - Inches(1)
    height = Inches(1)
    text_box = slide.shapes.add_textbox(left, top, width, height)
    text_frame = text_box.text_frame

    # Agregar texto al cuadro de texto
    p = text_frame.add_paragraph()
    p.text = text

    # Estilizar el texto
    p.alignment = PP_ALIGN.CENTER
    p.font.bold = True
    p.font.size = Pt(18)
    p.font.color.rgb = RGBColor(0, 51, 102)  # Color azul oscuro

# Crear una nueva presentación de PowerPoint
ppt = Presentation()

# # Ejemplo de uso
# add_image_with_text_box_to_presentation('/home/mandrade/Python-mejorado/Coordenadas-11-13.5S-7674W.jpg', ppt, 'Título de la imagen 1')
# add_image_with_text_box_to_presentation('/home/mandrade/Python-mejorado/Coordenadas-3.5-5.5S-7572W.jpg', ppt, 'Título de la imagen 2')
# add_image_with_text_box_to_presentation('/home/mandrade/Python-mejorado/Coordenadas-4-6S-8179.3W.jpg', ppt, 'Título de la imagen 3')
# add_image_with_text_box_to_presentation('/home/mandrade/Python-mejorado/ComparaHGT.jpg', ppt, 'Título de la imagen 4')

# # Guardar la presentación
# ppt.save('pruebaPPT.pptx')


In [ ]:
################### PROCESA METS #########################################################################                  
############                  PROCESA METS
################################################################################################
import xarray as xr
import pandas as pd
import numpy as np

# prueba
file_paths = [
    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
    '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
    '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
    '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
    '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
    '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
    '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc'
]
dates = [
    '2011-12-01 00:00',
    '2012-01-01 00:00',
    '2012-02-01 00:00',
    '2012-03-01 00:00',
    '2012-04-01 00:00',
    '2012-05-01 00:00',
    '2012-06-01 00:00',
]
datasets = []
# Procesar cada archivo y fecha correspondiente
for file_path, date in zip(file_paths, dates):
    ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
    date = pd.to_datetime(date)
    ds = ds.assign_coords({'tiempo': date})
    datasets.append(ds)
cc=xr.concat(datasets, dim='tiempo')
# fechas
frequencia = '6H'
date_range = pd.date_range(start=cc['tiempo'].min().values, end=cc['tiempo'].max().values, freq=frequencia)
metodo = 'linear'
interpolated_ds = cc.interp(tiempo=date_range) #, method=metodo)
interpolated_ds=interpolated_ds.sel(Time=0)
interpolated_ds = interpolated_ds.rename_dims({'tiempo': 'Time'})
interpolated_ds.to_netcdf(f"~/outputs/{frequencia}_{metodo}_compilado_met_COORDS.nc")
# interpolated_ds
# interpolated_ds=interpolated_ds.assign_coords(Time=('Time',interpolated_ds.tiempo.values)).drop_vars('tiempo') 
import xarray as xr
a=xr.open_dataset(f"~/outputs/{frequencia}_{metodo}_compilado_met_COORDS.nc")
a=a.assign_coords(Time=('Time',a.tiempo.values)).drop_vars('tiempo') # -> linea fiorela
a.to_netcdf(f"/home/mandrade/Python-mejorado/coords_{frequencia}_{metodo}_compilado_met_4.nc")

In [ ]:
############### Flxf       #############################################################
############################# Flxf               ##########################################
###########################################################################################

import pygrib
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
# rutas los archivos clim
rutas=[
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l01.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l02.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l03.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l04.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l05.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l06.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l07.fclm.1982.2010.grb2'
]

for each in rutas:
    ruta = each
    ruta_nameFile=ruta.split('/')[-1]
    
    # Mapeo de códigos de variable a sus nombres correspondientes
    variable_names = {
        37: 't',
        38: 'r',
        35: '10u',
        36: '10v',
        39: 'sp',
        6: 'ssw',
        7: 'sm',
        58: 'sm20',
        59: 'sm100',
        8: 'stl1',
        9: 'stl2',
        60: 'stl3',
        61: 'stl4',
        34: 'ci',
        33: 'lsm',
        5: 'skt',
        30:'prate',
        50:'pwater'
    }
    variables = [37,38,35,36,39,6,7,58,59,8,9,60,61,34,33,5,30,50]
    for variableNumber in variables:
        try:
            # Abrir el archivo GRIB y obtener datos
            grbs = pygrib.open(ruta)
            dataGrb = grbs[variableNumber].values
            lats, lons = grbs[variableNumber].latlons()
            stringFecha = ruta.split('/')[-1].split('.')[1]
            stringLed = ruta.split('/')[-1].split('.')[4][-1]
            tiempo = pd.to_datetime(f'2014-{stringFecha}-01 00:00') + pd.DateOffset(months=(int(stringLed) - 1))
            # print(f'fecha :{tiempo}')
            grbs.close()
    
            # Ajustar las dimensiones de las coordenadas
            latitudes = np.linspace(90, -90, lats.shape[0])
            longitudes = np.linspace(0, 360, lons.shape[1], endpoint=False)
            print("lat ", latitudes.shape, " lon ", longitudes.shape)
            # Crear el Dataset con xarray
            ds = xr.Dataset(
                {
                    variable_names[variableNumber]: (['latitud', 'longitud'], dataGrb),
                },
                coords={
                    'latitud': latitudes,
                    'longitud': longitudes,
                    'time': [tiempo],  # Se pasa tiempo como lista con un solo elemento
                },
            )
    
            # Corregir las coordenadas en el archivo NetCDF
            ds['latitud'].attrs['units'] = 'degrees_north'
            ds['longitud'].attrs['units'] = 'degrees_east'
            ds['latitud'].attrs['long_name'] = 'latitude'
            ds['longitud'].attrs['long_name'] = 'longitude'
    
            # Asignar atributos extend para la representación correcta en ncview
            ds['latitud'].attrs['actual_range'] = (-90.0, 90.0)
            ds['longitud'].attrs['actual_range'] = (-180, 180.0)
    
            # Guardar el archivo NetCDF
            ds.to_netcdf(f"{ruta_nameFile}_{variable_names[variableNumber]}.nc")
    
            # Graficar con imshow
            plt.imshow(ds[variable_names[variableNumber]], cmap='viridis', origin='upper', extent=[0, 360, -90, 90])
            plt.colorbar(label='Value')
            plt.title(f'Visualización de datos en GRIB para la variable {variable_names[variableNumber]}')
            plt.xlabel('Longitud')
            plt.ylabel('Latitud')
            plt.show()
    
        except Exception as e:
            print(f"Error en la variable {variable_names[variableNumber]}: {e}")
#combina los ds seleccionados

import pygrib
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

rutas=[
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l01.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l02.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l03.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l04.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l05.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l06.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/flxf.11.01.00.l07.fclm.1982.2010.grb2'
]


rutaConvertidos='/home/mandrade/outputs/FINAL/'

for ruta in rutas:
    ruta_nameFile = rutaConvertidos
    
    # Mapeo de códigos de variable a sus nombres correspondientes
    variable_names = {
        37: 't',
        38: 'r',
        35: '10u',
        36: '10v',
        39: 'sp',
        6: 'ssw',
        7: 'sm',
        58: 'sm20',
        59: 'sm100',
        8: 'stl1',
        9: 'stl2',
        60: 'stl3',
        61: 'stl4',
        34: 'ci',
        33: 'lsm',
        5: 'skt',
        30: 'prate',
        50: 'pwater'
    }

    variables = [37,38,35,36,39,6,7,58,59,8,9,60,61,34,33,5,30,50]
    datasets = []  # Lista para almacenar cada conjunto de datos
    
    for variableNumber in variables:
        try:
            # Abrir el archivo GRIB y obtener datos
            grbs = pygrib.open(ruta)
            dataGrb = grbs[variableNumber].values
            lats, lons = grbs[variableNumber].latlons()
            stringFecha = ruta.split('/')[-1].split('.')[1]
            stringLed = ruta.split('/')[-1].split('.')[4][-1]
            tiempo = pd.to_datetime(f'2014-{stringFecha}-01 00:00') + pd.DateOffset(months=(int(stringLed) - 1))
            # print(f'fecha :{tiempo}')
            grbs.close()
    
            # Ajustar las dimensiones de las coordenadas
            latitudes = np.linspace(90, -90, lats.shape[0])
            longitudes = np.linspace(0, 360, lons.shape[1], endpoint=False)
            print("lat ", latitudes.shape, " lon ", longitudes.shape)
            
            # Crear el Dataset con xarray
            ds = xr.Dataset(
                {
                    variable_names[variableNumber]: (['latitud', 'longitud'], dataGrb),
                },
                coords={
                    'latitud': latitudes,
                    'longitud': longitudes,
                    'time': [tiempo],  # Se pasa tiempo como lista con un solo elemento
                },
            )
    
            datasets.append(ds)  # Agregar cada Dataset a la lista
    
        except Exception as e:
            print(f"Error en la variable {variable_names[variableNumber]}: {e}")
    
    combined_ds = xr.merge(datasets)  # Combinar todos los datasets en uno solo
    combined_ds.to_netcdf(f"{ruta_nameFile}{ruta.split('/')[-1]}_convertido.nc")
    
    # Guardar el archivo NetCDF combinado
# INTERPOLAR DATOS PARA FLUJOS DE MANERA HORARIA
import xarray as xr
import pandas as pd

variable_names = [
        't',
        'r',
        '10u',
        '10v',
        'sp',
        'ssw',
        'sm',
        'sm20',
        'sm100',
        'stl1',
        'stl2',
        'stl3',
        'stl4',
        'ci',
        'lsm',
        'skt',
        'prate',
        'pwater'
]



# Función para seleccionar y procesar el conjunto de datos
def process_dataset(file_path, time_name, date_str):
    dataset = xr.open_dataset(file_path)
    selec_ds = dataset[variable_names]
    # selec_ds = selec_ds.rename({'tiempo': time_name})
    date = pd.to_datetime(date_str)
    selec_ds = selec_ds.assign_coords({'time': date})
    return selec_ds
# Lista de archivos y fechas correspondientes

rutaConvertidos='/home/mandrade/outputs/FINAL/'

file_paths = [
    f'{rutaConvertidos}flxf.11.01.00.l01.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}flxf.11.01.00.l02.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}flxf.11.01.00.l03.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}flxf.11.01.00.l04.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}flxf.11.01.00.l05.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}flxf.11.01.00.l06.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}flxf.11.01.00.l07.fclm.1982.2010.grb2_convertido.nc'
]
dates = [
    '2011-12-01 00:00',
    '2012-01-01 00:00',
    '2012-02-01 00:00',
    '2012-03-01 00:00',
    '2012-04-01 00:00',
    '2012-05-01 00:00',
    '2012-06-01 00:00'
]
# Lista para almacenar los conjuntos de datos seleccionados y procesados
selected_datasets = []
# Procesar cada archivo y fecha correspondiente
for i, file_path in enumerate(file_paths):
    time_name = str(i)
    date_str = dates[i]
    ds = process_dataset(file_path, time_name, date_str)
    selected_datasets.append(ds)
# Combinar los conjuntos de datos en la dimensión "tiempo"
merged_ds = xr.concat(selected_datasets, dim='time')
# Obtener el rango de fechas y realizar la interpolación
frequencia = '6H'
date_range = pd.date_range(start=merged_ds['time'].min().values, end=merged_ds['time'].max().values, freq=frequencia)
metodo = 'cubic'
interpolated_ds = merged_ds.interp(time=date_range) #, method=metodo)
# Guardar el conjunto de datos interpolado en un archivo NetCDF
interpolated_ds.to_netcdf(f"/home/mandrade/Python-mejorado/final_flxf_{frequencia}_{metodo}_compilado_met.nc")

In [ ]:
#############       PGBF  ##################################################
#############################       PGBF                         ##########################
###########################################################################################


###
import pygrib
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

rutas=[
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l01.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l02.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l03.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l04.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l05.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l06.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l07.fclm.1982.2010.grb2'
]


rutaConvertidos='/home/mandrade/outputs/FINAL/'

for each in rutas:
    ruta = each
    ruta_nameFile=ruta.split('/')[-1]
    
    # Mapeo de códigos de variable a sus nombres correspondientes
    variable_names = {
        429: 't',
        431: '10u',
        432: '10v',
        430: 'r',
        357: 'gh',
        1: 'msl',
        369: 'tprecip',
        419:'pwater'
    }
    
    variables = [429,431,432,430,357,1,369,419]
    
    for variableNumber in variables:
        try:
            # Abrir el archivo GRIB y obtener datos
            grbs = pygrib.open(ruta)
            dataGrb = grbs[variableNumber].values
            lats, lons = grbs[variableNumber].latlons()
            stringFecha = ruta.split('/')[-1].split('.')[1]
            stringLed = ruta.split('/')[-1].split('.')[4][-1]
            tiempo = pd.to_datetime(f'2014-{stringFecha}-01 00:00') + pd.DateOffset(months=(int(stringLed) - 1))
            print(f'fecha :{tiempo}')
            grbs.close()
    
            # Ajustar las dimensiones de las coordenadas
            latitudes = np.linspace(90, -90, lats.shape[0])
            longitudes = np.linspace(0, 360, lons.shape[1], endpoint=False)
            print("lat ", latitudes.shape, " lon ", longitudes.shape)
            # Crear el Dataset con xarray
            ds = xr.Dataset(
                {
                    variable_names[variableNumber]: (['latitud', 'longitud'], dataGrb),
                },
                coords={
                    'latitud': latitudes,
                    'longitud': longitudes,
                    'time': [tiempo],  # Se pasa tiempo como lista con un solo elemento
                },
            )
    
            # Corregir las coordenadas en el archivo NetCDF
            ds['latitud'].attrs['units'] = 'degrees_north'
            ds['longitud'].attrs['units'] = 'degrees_east'
            ds['latitud'].attrs['long_name'] = 'latitude'
            ds['longitud'].attrs['long_name'] = 'longitude'
    
            # Asignar atributos extend para la representación correcta en ncview
            ds['latitud'].attrs['actual_range'] = (-90.0, 90.0)
            ds['longitud'].attrs['actual_range'] = (-180, 180.0)
    
            # Guardar el archivo NetCDF
            ds.to_netcdf(f"{rutaConvertidos}{ruta_nameFile}_{variable_names[variableNumber]}.nc")
    
            # Graficar con imshow
            plt.imshow(ds[variable_names[variableNumber]], cmap='viridis', origin='upper', extent=[0, 360, -90, 90])
            plt.colorbar(label='Value')
            plt.title(f'Visualización de datos en GRIB para la variable {variable_names[variableNumber]}')
            plt.xlabel('Longitud')
            plt.ylabel('Latitud')
            plt.savefig(f'FLXF_{variable_names[variableNumber]}.png')
            add_image_with_title_to_presentation(f'FLXF_{variable_names[variableNumber]}.png', ppt, f'{variable_names[variableNumber]}')
            plt.show()
    
        except Exception as e:
            print(f"Error en la variable {variable_names[variableNumber]}: {e}")
#combina los ds seleccionados

import pygrib
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd

rutas=[
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l01.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l02.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l03.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l04.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l05.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l06.fclm.1982.2010.grb2',
    '/home/mandrade/Data/CLIM_8129_00/pgbf.11.01.00.l07.fclm.1982.2010.grb2'
]
rutaConvertidos='/home/mandrade/outputs/FINAL/'
for each in rutas:
    ruta = each
    ruta_nameFile=ruta.split('/')[-1]
    
    # Mapeo de códigos de variable a sus nombres correspondientes
    variable_names = {
        429: 't',
        431: '10u',
        432: '10v',
        430: 'r',
        357: 'gh',
        1:   'msl',
        369: 'tprecip',
        419:'pwater'
    }
    
    variables = [429,431,432,430,357,1,369,419]
    
    datasets = []  # Lista para almacenar cada conjunto de datos
    
    for variableNumber in variables:
        try:
            # Abrir el archivo GRIB y obtener datos
            grbs = pygrib.open(ruta)
            dataGrb = grbs[variableNumber].values
            lats, lons = grbs[variableNumber].latlons()
            stringFecha = ruta.split('/')[-1].split('.')[1]
            stringLed = ruta.split('/')[-1].split('.')[4][-1]
            tiempo = pd.to_datetime(f'2014-{stringFecha}-01 00:00') + pd.DateOffset(months=(int(stringLed) - 1))
            print(f'fecha :{tiempo}')
            grbs.close()
    
            # Ajustar las dimensiones de las coordenadas
            latitudes = np.linspace(90, -90, lats.shape[0])
            longitudes = np.linspace(0, 360, lons.shape[1], endpoint=False)
            print("lat ", latitudes.shape, " lon ", longitudes.shape)
            
            # Crear el Dataset con xarray
            ds = xr.Dataset(
                {
                    variable_names[variableNumber]: (['latitud', 'longitud'], dataGrb),
                },
                coords={
                    'latitud': latitudes,
                    'longitud': longitudes,
                    'time': [tiempo],  # Se pasa tiempo como lista con un solo elemento
                },
            )
    
            datasets.append(ds)  # Agregar cada Dataset a la lista
    
        except Exception as e:
            print(f"Error en la variable {variable_names[variableNumber]}: {e}")
    
    combined_ds = xr.merge(datasets)  # Combinar todos los datasets en uno solo
    combined_ds.to_netcdf(f"~/pgbf_convertido.nc")
    
    # Guardar el archivo NetCDF combinado
# INTERPOLAR DATOS PARA PGBF
import xarray as xr
import pandas as pd

variables_nombres_flx = [
    't',
    '10u',
    '10v',
    'r',
    'gh',
    'msl',
    'tprecip',
    'pwater'
]
rutaConvertidos='/home/mandrade/outputs/FINAL/'
# Función para seleccionar y procesar el conjunto de datos
def process_dataset(file_path, time_name, date_str):
    dataset = xr.open_dataset(file_path)
    selec_ds = dataset[variables_nombres_flx]
    # selec_ds = selec_ds.rename({'tiempo': time_name})
    date = pd.to_datetime(date_str)
    selec_ds = selec_ds.assign_coords({'time': date})
    return selec_ds
# Lista de archivos y fechas correspondientes
file_paths = [
    f'{rutaConvertidos}pgbf.11.01.00.l01.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}pgbf.11.01.00.l02.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}pgbf.11.01.00.l03.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}pgbf.11.01.00.l04.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}pgbf.11.01.00.l05.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}pgbf.11.01.00.l06.fclm.1982.2010.grb2_convertido.nc',
    f'{rutaConvertidos}pgbf.11.01.00.l07.fclm.1982.2010.grb2_convertido.nc'
]
anho='2014'
dates = [
    '2011-12-01 00:00',
    '2012-01-01 00:00',
    '2012-02-01 00:00',
    '2012-03-01 00:00',
    '2012-04-01 00:00',
    '2012-05-01 00:00',
    '2012-06-01 00:00'
]
# Lista para almacenar los conjuntos de datos seleccionados y procesados
selected_datasets = []
# Procesar cada archivo y fecha correspondiente
for i, file_path in enumerate(file_paths):
    time_name = str(i)
    date_str = dates[i]
    ds = process_dataset(file_path, time_name, date_str)
    selected_datasets.append(ds)
# Combinar los conjuntos de datos en la dimensión "tiempo"
merged_ds = xr.concat(selected_datasets, dim='time')
# Obtener el rango de fechas y realizar la interpolación
frequencia = '6H'
date_range = pd.date_range(start=merged_ds['time'].min().values, end=merged_ds['time'].max().values, freq=frequencia)
metodo = 'linear'
interpolated_ds = merged_ds.interp(time=date_range) #, method=metodo)
# Guardar el conjunto de datos interpolado en un archivo NetCDF
vmuda1=f"~/Diciembre/outputs/final_pgbf_{frequencia}_{metodo}_compilado_met.nc"
interpolated_ds.to_netcdf(f"/home/mandrade/Python-mejorado/final_pgbf_{frequencia}_{metodo}_compilado_met.nc")


## ERA5 (Reanálisis Climático de la Era 5)


- Resolución Espacial: 0.25 grados (~27 km)
- Resolución Temporal: 6 Horas
- Periodo 81-2010 en el cluster en la ruta:
<span style="color:red">- /data/datos/ERA/Clim/data</span>


## CFS (Sistema de Pronóstico Climático)


- Resolución Espacial: 0.5 grados (~55 km)
- Resolución Temporal: Mensual
- Período de Pronóstico: Hasta varios meses en el futuro , se esta tomando 7 leads a partir del mes 

## DATOS CFS CORREGIDOS


- Resolución Espacial: 0.25 grados (~27 km)
- Resolución Temporal interpolada: 6 Horas


In [ ]:
#SIMPLIFICA GENERAR METS EN UNO SOLO 
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import cartopy.io.shapereader

descripciones_var = {
    'TT': 'Temperatura superficial del aire a 2m (°C)',
    'PRES': 'Presión a nivel del mar (hPa)',
    'RH': 'Humedad Relativa (%)',
    'GHT': 'Altura Geopotencial (m)',
    'UU': 'Componente zonal U del viento a 10m',
    'VV': 'Componente meridional V del viento a 10m'
}

def generaMapasMETS(var, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]

    # Asegurarse de que las dimensiones de las coordenadas sean compatibles
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")

    fig, axs = plt.subplots(3, 3, figsize=(15, 12), subplot_kw={'projection': ccrs.PlateCarree()})
    valid_subplots = [ax for ax in axs.flat[:len(meses_dict)]]
    cbar_ax = fig.add_axes([0.15, 0.03, 0.7, 0.02])

    for i, mes in enumerate(meses_dict.keys()):
        ax = valid_subplots[i]
        mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
        im = mes_data[var].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm')
        ax.set_title(f'Mes: {meses_dict[mes]}')
        gl = ax.gridlines(draw_labels=True, linestyle='--')
        gl.xlabels_top = False; gl.ylabels_right = False
        ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')

    cbar = plt.colorbar(im, cax=cbar_ax, orientation='horizontal', shrink=0.6, pad=0.05)
    cbar.set_label(f'{descripciones_var[var]}')

    for i in range(len(meses_dict) + 1, 10):
        plt.subplot(3, 3, i).axis('off')

    plt.subplots_adjust(top=0.92, hspace=0.5)
    plt.tight_layout(rect=[0, 0.05, 1, 0.97])
    plt.savefig(f'Muda.jpg')
    plt.show()

generaMapasMETS('TT')
generaMapasMETS('PRES')
generaMapasMETS('RH')
generaMapasMETS('GHT')
generaMapasMETS('UU', coord_lon='XLONG_U', coord_lat='XLAT_U', stag_west='west_east_stag', stag_north='south_north')
generaMapasMETS('VV', coord_lon='XLONG_V', coord_lat='XLAT_V', stag_west='west_east', stag_north='south_north_stag')


In [ ]:
###### MAPAS DE TEMP
#### temperatura ######
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale

# Configurar la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")

descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
# Diccionario para mapear los números de los meses a sus nombres
var_min = -10  # Define el valor mínimo deseado
var_max = 30 
def generaMapasMETS_en_subplot(ax, var, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    mes_data = mes_data - 273.15
    im = mes_data[var].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.025)
    cbar.set_label('Temperatura del aire (°C) CFS - Corregido')  # Etiqueta de la barra de color
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

###### funcion genera FNL en columna 1
def generaMapasFNL_en_subplot(ax, mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-tmp.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp['TMPprs'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev=100000).mean(dim='time')
    mes_temp = mes_temp - 273.15  # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.025)
    cbar.set_label('Temperatura del aire (°C) CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 10))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 10))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
##########
def generaMapasFLX_en_subplot(ax, mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    flx = flx['t'].sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2),time=flx['time.month'] == int(mes)).mean(dim='time')
    flx = abs(flx - 273.15)  # Convertir a grados Celsius
    im = flx.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.05)
    cbar.set_label('Temperatura del aire (°C) CLIMATOLOGIA - CFS DATACRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 10))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 10))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
# Crear el subplot 6x2
fig, axs = plt.subplots(7, 3, figsize=(18, 24), subplot_kw={'projection': ccrs.PlateCarree()})
# Asumiendo que quieres la variable 'TT' para diferentes meses
variableMet = 'TT'
variableFNL='TMPprs'
variableFLX='t'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        im0=generaMapasFLX_en_subplot(axs[i, 2], mes)
        im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
# Mostrar el subplot
plt.show()

In [ ]:
# ###### funcion genera FNL en columna 2
# #### temperatura ######
# import xarray as xr
# import matplotlib.pyplot as plt
# import pandas as pd
# import numpy as np
# import matplotlib.dates as mdates
# import cartopy.crs as ccrs
# import cartopy
# import warnings
# import locale

# # Configurar la localización en español
# locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# # Evitar las advertencias de deprecación de cartopy
# warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
# warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")

# descripciones_var = {
#         'TT': 'Temperatura superficial del aire a 2m (°C)',
#         'PRES': 'Presión a nivel del mar (hPa)',
#         'RH': 'Humedad Relativa (%)',
#         'GHT': 'Altura Geopotencial (m)',
#         'UU': 'Componente zonal U del viento a 10m ',
#         'VV': 'Componente meridional V del viento a 10m '
#     }
# # Diccionario para mapear los números de los meses a sus nombres
# var_min = -10  # Define el valor mínimo deseado
# var_max = 30 
# def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
#     meses_dict = {
#         '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
#         '04': 'Abril', '05': 'Mayo', '06': 'Junio'
#     }
#     ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
#     longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
#     latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
#     if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
#         ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
#     else:
#         raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
#     mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
#     mes_data = mes_data - 273.15
#     im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
#     ax.set_title(f'Mes: {meses_dict[mes]}')
#     gl = ax.gridlines(draw_labels=True, linestyle='--')
#     cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.05)
#     cbar.set_label('Temperatura del aire (°C) CFS - Corregido')  # Etiqueta de la barra de color
#     gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
#     gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
#     gl.xlabels_top = False; gl.ylabels_right = False
#     ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
#     return im

# ###### funcion genera FNL en columna 1
# def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
#     temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-tmp.nc')
#     meses_dict = {
#         '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
#         '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
#     mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev=100000).mean(dim='time')
#     mes_temp = mes_temp - 273.15  # Convertir a grados Celsius
#     im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
#     ax.set_title(f'Mes: {meses_dict[mes]}')
#     gl = ax.gridlines(draw_labels=True, linestyle='--')
#     cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.05)
#     cbar.set_label('Temperatura del aire (°C) CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
#     gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
#     gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
#     gl.xlabels_top = False
#     gl.ylabels_right = False
#     ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
#     return im
# ##########
# def generaMapasFLX_en_subplot(ax, var_flx,mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
#     flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
#     meses_dict = {
#         '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
#         '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
#     flx = flx[var_flx].sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2),time=flx['time.month'] == int(mes)).mean(dim='time')
#     flx = abs(flx - 273.15)  # Convertir a grados Celsius
#     im = flx.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
#     ax.set_title(f'Mes: {meses_dict[mes]}')
#     gl = ax.gridlines(draw_labels=True, linestyle='--')
#     cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.05)
#     cbar.set_label('Temperatura del aire (°C) CLIMATOLOGIA - CFS DATACRUDA')  # Etiqueta de la barra de color
#     gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
#     gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
#     gl.xlabels_top = False
#     gl.ylabels_right = False
#     ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
#     return im
# # Crear el subplot 6x2
# fig, axs = plt.subplots(7, 3, figsize=(18, 24), subplot_kw={'projection': ccrs.PlateCarree()})
# # Asumiendo que quieres la variable 'TT' para diferentes meses
# variableMet = 'TT'
# variableFNL='TMPprs'
# variableFLX='t'
# meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
# for i, mes in enumerate(meses):
#     if i < 7:  # Asegurarse de no exceder el número de filas
#         im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
#         im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
#         im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
#         # Si necesitas un colorbar, puedes agregarlo aquí
# # Ajustar los espacios entre los gráficos
# plt.tight_layout()
# # Mostrar el subplot
# plt.show()

# MAPAS COMPARATIVOS DE DATA CRUDA VS FNL VS CFS VS ERA5 EN CASO DE VIENTOS

- DATOS DE ERA 5 TOMADOS DE PROMEDIOS MENSUALES EN /DATA/DATOS/

In [ ]:
#### temperatura ###### FUNCIONA
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '}
var_min = -10  
var_max = 30
contour_levels = np.arange(-10,30, 5)

def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {'12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    mes_data=mes_data-273.15
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(),levels=contour_levels, add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=1)
    ax.set_title(f'{meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.4, pad=0.15)
    cbar.set_label('CFS INTERPOLADO')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    mes_data = mes_data - 273.15
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(),levels=contour_levels, add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=1)
    ax.set_title(f'{meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.4, pad=0.15)
    cbar.set_label('CFS - Corregido')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-tmp.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev=100000).mean(dim='time')
    mes_temp = mes_temp - 273.15  # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), levels=contour_levels,add_colorbar=False, cmap='coolwarm',vmin=var_min,vmax=var_max)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'{meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.4, pad=0.15)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

fig, axs = plt.subplots(7, 3, figsize=(15, 28), subplot_kw={'projection': ccrs.PlateCarree()})
variableMet = 'TT'
variableFNL='TMPprs'
variableFLX='t'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        im0 = generaMapasMETScompilado(axs[i,2], variableMet, mes)
        im = generaMapasMETS_en_subplot(axs[i,1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
plt.tight_layout(pad=1.0)
fig.suptitle('Temperatura del aire (°C)', fontsize=16, y=1.02)
plt.savefig(f'ComparaTemp.jpg')
add_image_with_title_to_presentation(f'ComparaTemp.jpg', ppt, 'TEMPERATURA DEL AIRE 2m')
plt.show()

In [ ]:
#### PRES ###### FUNCIONA
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
# Configurar la localización en español
vmin=60000
vmax=102000
contour_levels = np.arange(vmin,vmax,(vmax-vmin)/5)
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
# Diccionario para mapear los números de los meses a sus nombres
def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS DATA CRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
    
def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    # mes_data = mes_data - 273.15
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS INTERPOLADO')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

###### funcion genera FNL en columna 1
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-pre.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes)).mean(dim='time')
    # mes_temp = mes_temp - 273.15  # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

fig, axs = plt.subplots(7, 3, figsize=(15, 28), subplot_kw={'projection': ccrs.PlateCarree()})
variableMet = 'PRES'
variableFNL='PRESmsl'
variableFLX='sp'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        # im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
        im0 = generaMapasMETScompilado(axs[i, 2], variableMet, mes)
        im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
# Mostrar el subplot
plt.suptitle('Presion de superficie (Pa)', fontsize=16, y=1.02)
plt.savefig(f'ComparaPres.jpg')
add_image_with_title_to_presentation(f'ComparaPres.jpg', ppt, 'PRESION DE SUPERFICIE')
plt.show()

In [ ]:
#### PRES ###### FUNCIONA
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
# Configurar la localización en español
vmin=98000
vmax=102000
contour_levels = np.arange(vmin,vmax,(vmax-vmin)/5)
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
# Diccionario para mapear los números de los meses a sus nombres
def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS DATA CRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
    
def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    # mes_data = mes_data - 273.15
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS INTERPOLADO')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

###### funcion genera FNL en columna 1
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-pre.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes)).mean(dim='time')
    # mes_temp = mes_temp - 273.15  # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

fig, axs = plt.subplots(7, 3, figsize=(15, 28), subplot_kw={'projection': ccrs.PlateCarree()})
variableMet = 'PRES'
variableFNL='PRESmsl'
variableFLX='sp'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        # im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
        im0 = generaMapasMETScompilado(axs[i, 2], variableMet, mes)
        im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
# Mostrar el subplot
plt.suptitle('Presion de superficie (Pa)', fontsize=16, y=1.02)
plt.savefig(f'ComparaPres.jpg')
add_image_with_title_to_presentation(f'ComparaPres.jpg', ppt, 'PRESION DE SUPERFICIE')
plt.show()

In [ ]:
#### HUMEDAD RELATIVA ###### FUNCIONA
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
# Configurar la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
vmin=30
vmax=100
contour_levels = np.arange(vmin,vmax,(vmax-vmin)/5)
# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
# Diccionario para mapear los números de los meses a sus nombres
def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    # mes_data = mes_data - 273.15
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS INTERPOLADO')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
#### GENERA MAPA METS CRUDO ################### ERROR
def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS DATA CRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im


###### funcion genera FNL en columna 1
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-rhp.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev_3=100000).mean(dim='time')
    # mes_temp = mes_temp - 273.15  # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)    
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
##########
def generaMapasFLX_en_subplot(ax, var_flx,mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    flx = flx[var_flx].sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2),time=flx['time.month'] == int(mes)).mean(dim='time')
    # flx = abs(flx - 273.15)  # Convertir a grados Celsius
    im = flx.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS FLXF')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
# Crear el subplot 6x2
fig, axs = plt.subplots(7, 3, figsize=(15, 28), subplot_kw={'projection': ccrs.PlateCarree()})
# Asumiendo que quieres la variable 'TT' para diferentes meses
variableMet = 'RH'
variableFNL='RHprs'
variableFLX='r'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        # im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
        im0 = generaMapasMETScompilado(axs[i, 2], variableMet, mes)
        im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
# Mostrar el subplot
plt.suptitle('Humedad Relativa (%)', fontsize=16, y=1.02)
plt.savefig(f'ComparaHR.jpg')
add_image_with_title_to_presentation(f'ComparaHR.jpg', ppt, 'HUMEDAD RELATIVA (%)')
plt.show()

In [ ]:
#### HGT ###### FUNCIONA
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale

vmin=0
vmax=200
contour_levels = np.arange(vmin,vmax,(vmax-vmin)/5)
# Configurar la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")

descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
# Diccionario para mapear los números de los meses a sus nombres

def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(),  add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS - INTERPOLADO')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS DATA CRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

###### funcion genera FNL en columna 1
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-hgt.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev=100000).mean(dim='time')
    # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)        
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
##########
# Crear el subplot 6x2
fig, axs = plt.subplots(7, 3, figsize=(15, 28), subplot_kw={'projection': ccrs.PlateCarree()})
# Asumiendo que quieres la variable 'TT' para diferentes meses
variableMet = 'GHT'
variableFNL='HGTprs'
variableFLX='gh'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        # im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
        im0 = generaMapasMETScompilado(axs[i, 2], variableMet, mes)
        im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
plt.suptitle('Altura Geopotencial (m)', fontsize=16, y=1.02)
plt.savefig(f'ComparaHGT.jpg')
add_image_with_title_to_presentation(f'ComparaHGT.jpg', ppt, 'ALTURA GEOPOTENCIAL (m)')
# Mostrar el subplot
plt.show()

In [ ]:
#### HGT ##### COMPARATIVO 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale

vmin=0
vmax=5000
contour_levels = np.arange(vmin,vmax,(vmax-vmin)/5)
# Configurar la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")

descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
# Diccionario para mapear los números de los meses a sus nombres

def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(),  add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.12)
    cbar.set_label('Altura geopotencial (m) CFS - Corregido')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_M', coord_lat='XLAT_M', stag_west='west_east', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)    
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS DATA CRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

###### funcion genera FNL en columna 1
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-hgt.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev=100000).mean(dim='time')
    # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)    
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.3, pad=0.12)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
##########
# Crear el subplot 6x2
fig, axs = plt.subplots(7, 3, figsize=(15, 28), subplot_kw={'projection': ccrs.PlateCarree()})
# Asumiendo que quieres la variable 'TT' para diferentes meses
variableMet = 'GHT'
variableFNL='HGTprs'
variableFLX='gh'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        # im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
        im0 = generaMapasMETScompilado(axs[i, 2], variableMet, mes)
        im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
plt.suptitle('Altura Geopotencial (m)', fontsize=16, y=1.02)
plt.savefig(f'ComparaHGT.jpg')
add_image_with_title_to_presentation(f'ComparaHGT.jpg', ppt, 'ALTURA GEOPOTENCIAL (m)')
# Mostrar el subplot
plt.show()

In [ ]:
#### UU ###### FUNCIONA
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
# Configurar la localización en español
vmin=-10
vmax=10
contour_levels = np.arange(vmin,vmax,(vmax-vmin)/5)
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '}

def generaMapaERA5_en_subplot(ax,var_era5 , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    latitud = slice(latitud,latitud2)
    longitud = slice(longitud,longitud2)
    era581 = xr.open_mfdataset('/data/datos/ERA/Clim/data/era_*.nc')
    era581 = era581.sel(latitude=latitud, longitude=longitud)
    mes_data = era581.sel(time=era581['time.month'] == int(mes)).mean(dim='time')
    im = mes_data[var_era5].plot(ax=ax, transform=ccrs.PlateCarree(),  add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(mes_data.longitude, mes_data.latitude, mes_data[var_era5], levels=contour_levels, colors='k', linewidths=0.5)
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA ERA5')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_U', coord_lat='XLAT_U', stag_west='west_east_stag', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(),  add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)   
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA INTERPOLADO')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

# '/home/mandrade/Python-mejorado/ncCompilado.nc'
def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_U', coord_lat='XLAT_U', stag_west='west_east_stag', stag_north='south_north'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9) 
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS DATA CRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-ugr.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev=100000).mean(dim='time')
    # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)    
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'{meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
fig, axs = plt.subplots(7, 4, figsize=(15, 30), subplot_kw={'projection': ccrs.PlateCarree()})
# Asumiendo que quieres la variable 'TT' para diferentes meses
variableMet = 'UU'
variableFNL='UGRDprs'
variableFLX='10u'
variableERA5='u10'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        # im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
        im = generaMapasMETScompilado(axs[i, 2], variableMet, mes)
        im1 = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
        im3= generaMapaERA5_en_subplot(axs[i,3],variableERA5,mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
fig.suptitle('Velocidad del viento U (m/s)', fontsize=16, y=1.02)
plt.savefig(f'ComparaUU.jpg')
add_image_with_title_to_presentation(f'ComparaUU.jpg', ppt, 'Viento local U (m/s)')
# Mostrar el subplot
plt.show()

In [ ]:
#### VV ###### FUNCIONA
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
vmin=-10
vmax=10
contour_levels = np.arange(vmin,vmax,(vmax-vmin)/5)
# Configurar la localización en español
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
# Evitar las advertencias de deprecación de cartopy
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = {
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '}

def generaMapaERA5_en_subplot(ax,var_era5 , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    latitud = slice(latitud,latitud2)
    longitud = slice(longitud,longitud2)
    era581 = xr.open_mfdataset('/data/datos/ERA/Clim/data/era_*.nc')
    era581 = era581.sel(latitude=latitud, longitude=longitud)
    mes_data = era581.sel(time=era581['time.month'] == int(mes)).mean(dim='time')
    im = mes_data[var_era5].plot(ax=ax, transform=ccrs.PlateCarree(),  add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(mes_data.longitude, mes_data.latitude, mes_data[var_era5], levels=contour_levels, colors='k', linewidths=0.5)
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA ERA5')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasMETS_en_subplot(ax, var_met, mes, coord_lon='XLONG_V', coord_lat='XLAT_V', stag_west='west_east', stag_north='south_north_stag'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('~/coords_6H_linear_compilado_met_2.nc').sel(num_metgrid_levels=0)
    longitudes = ds2[coord_lon].isel(Time=1).values[0, :]
    latitudes = ds2[coord_lat].isel(Time=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(Time=ds2['Time.month'] == int(mes)).mean(dim='Time')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(),  add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)    
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS INTERPOLADO')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im

# '/home/mandrade/Python-mejorado/ncCompilado.nc'
def generaMapasMETScompilado(ax, var_met, mes, coord_lon='XLONG_V', coord_lat='XLAT_V', stag_west='west_east', stag_north='south_north_stag'):
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'
    }
    ds2 = xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc').sel(num_metgrid_levels=0)
    ds2=ds2.sel(Time=0)
    longitudes = ds2[coord_lon].isel(tiempo=1).values[0, :]
    latitudes = ds2[coord_lat].isel(tiempo=1).values[:, 0]
    if longitudes.shape[0] == ds2.dims[stag_west] and latitudes.shape[0] == ds2.dims[stag_north]:
        ds2 = ds2.assign_coords({stag_west: longitudes, stag_north: latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    mes_data = ds2.sel(tiempo=ds2['tiempo.month'] == int(mes)).mean(dim='tiempo')
    im = mes_data[var_met].plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False, cmap='coolwarm',vmin=vmin,vmax=vmax)
    cs = ax.contour(longitudes, latitudes, mes_data[var_met], linewidths=0.4, levels=contour_levels, colors='black', transform=ccrs.PlateCarree())
    ax.clabel(cs, inline=True, fontsize=9)    
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CFS DATA CRUDA')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False; gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
def generaMapasFNL_en_subplot(ax,var_fnl , mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
    temp = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-vgr.nc')
    meses_dict = {
        '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
        '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
    mes_temp = temp[var_fnl].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),time=temp['time.month'] == int(mes),lev=100000).mean(dim='time')
    # Convertir a grados Celsius
    im = mes_temp.plot(ax=ax, transform=ccrs.PlateCarree(), add_colorbar=False,cmap='coolwarm',vmin=vmin,vmax=vmax)
    if contour_levels is not None:
        cs=ax.contour(mes_temp.lon, mes_temp.lat, mes_temp, levels=contour_levels, colors='k', linewidths=0.5)    
    ax.clabel(cs, inline=True, fontsize=9)
    ax.set_title(f'Mes: {meses_dict[mes]}')
    gl = ax.gridlines(draw_labels=True, linestyle='--')
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
    cbar.set_label('CLIMATOLOGIA FNL')  # Etiqueta de la barra de color
    gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
    gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
    gl.xlabels_top = False
    gl.ylabels_right = False
    ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
    return im
##########
# def generaMapasFLX_en_subplot(ax, var_flx,mes,latitud=6, latitud2=-24, longitud=360-100, longitud2=360-65):
#     flx = xr.open_dataset('/home/mandrade/Python-mejorado/final_pgbf_6H_linear_compilado_met.nc')
#     meses_dict = {
#         '12': 'Diciembre', '01': 'Enero', '02': 'Febrero', '03': 'Marzo', 
#         '04': 'Abril', '05': 'Mayo', '06': 'Junio'}
#     flx = flx[var_flx].sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2),time=flx['time.month'] == int(mes)).mean(dim='time')
#      # Convertir a grados Celsius
#     im = flx.plot(ax=ax, transform=ccrs.PlateCarree(),add_colorbar=False, cmap='coolwarm')
#     ax.set_title(f'Mes: {meses_dict[mes]}')
#     gl = ax.gridlines(draw_labels=True, linestyle='--')
#     cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.5, pad=0.12)
#     cbar.set_label('CLIMATOLOGIA CFS DATACRUDA')  # Etiqueta de la barra de color
#     gl.xlocator = plt.FixedLocator(np.arange(-180, 181, 5))
#     gl.ylocator = plt.FixedLocator(np.arange(-90, 91, 5))
#     gl.xlabels_top = False
#     gl.ylabels_right = False
#     ax.add_geometries(cartopy.io.shapereader.Reader('/home/mandrade/Python-mejorado/Countries/ne_10m_admin_0_countries_lakes.shp').geometries(), ccrs.PlateCarree(), edgecolor='black', facecolor='none')
#     return im
# Crear el subplot 6x2
fig, axs = plt.subplots(7, 4, figsize=(18, 24), subplot_kw={'projection': ccrs.PlateCarree()})
# Asumiendo que quieres la variable 'TT' para diferentes meses
variableMet = 'VV'
variableFNL='VGRDprs'
variableFLX='10v'
variableERA5='v10'
meses = ['12','01', '02', '03', '04', '05', '06']  # Ejemplo de meses
for i, mes in enumerate(meses):
    if i < 7:  # Asegurarse de no exceder el número de filas
        # im0=generaMapasFLX_en_subplot(axs[i, 2], variableFLX,mes)
        im = generaMapasMETScompilado(axs[i, 2], variableMet, mes)
        im = generaMapasMETS_en_subplot(axs[i, 1], variableMet, mes)
        im2= generaMapasFNL_en_subplot(axs[i,0],variableFNL,mes)
        im3= generaMapaERA5_en_subplot(axs[i,3],variableERA5,mes)
        # Si necesitas un colorbar, puedes agregarlo aquí
# Ajustar los espacios entre los gráficos
plt.tight_layout()
fig.suptitle('Componente meridional V (m/s)', fontsize=16, y=1.02)
plt.savefig(f'ComparaUU.jpg')
add_image_with_title_to_presentation(f'ComparaUU.jpg', ppt, 'Viento local U (m/s)')
# Mostrar el subplot
plt.show()

In [ ]:
ppt.save('TempPPT.pptx')

In [ ]:
################### PROCESA METS #########################################################################                  
############                  PROCESA METS
################################################################################################
def getdataTVmets(l1=-20,l2=0,w1=-90,w2=-70):
    import xarray as xr
    import pandas as pd
    import numpy as np
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    # Procesar cada archivo y fecha correspondiente
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    longitudes = ds_concat['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
    # Asignar las coordenadas
    if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north']:
        ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['PRES'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed.sel(tiempo=ds_renamed.tiempo[0]).plot()
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    metsDCValues=ds_renamed.values
    metsDCTime=pd.to_datetime(ds_renamed.tiempo)+pd.DateOffset(years=2)
    return metsDCTime, metsDCValues



getdataPROCESADA()
metsDCTime,metsDCValues = getdataTVmets()
plt.figure()
plt.scatter(metsDCTime,metsDCValues,s=85,label='DATA CRUDA')
plt.legend()

In [ ]:

# def getMetsProc(l1=-10,l2=0, w1=-90,w2=-80):
#     import xarray as xr
#     import numpy as np
#     import matplotlib.pyplot as plt
#     import pandas as pd
#     dsMETSPROC=xr.open_dataset('/home/mandrade/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
#     longitudes = dsMETSPROC['XLONG_M'].isel( Time=0).values[0, :]
#     latitudes = dsMETSPROC['XLAT_M'].isel(Time=0).values[:, 0]
#     if longitudes.shape[0] == dsMETSPROC.dims['west_east'] and latitudes.shape[0] == dsMETSPROC.dims['south_north']:
#         dsMETSPROC = dsMETSPROC.assign_coords({'west_east': longitudes, 'south_north': latitudes})
#     else:
#         raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
#     dsMETSPROC = dsMETSPROC.sel(num_metgrid_levels=0)
#     dsMETSPROC = dsMETSPROC.sel(south_north=slice(l1,l2),west_east=slice(w1,w2))
#     dsMETSPROC = dsMETSPROC.mean(['west_east','south_north'])
#     valuesMETSPROC=dsMETSPROC['TT'].values
#     timeMETSPROC=dsMETSPROC['TT'].Time
#     return valuesMETSPROC,timeMETSPROC

# def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
#     import xarray as xr
#     import pandas as pd
#     import numpy as np
#     # prueba
#     file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
#         '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
#         '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
#         '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
#         '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
#         '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
#         '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
#     dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
#     datasets = []
#     for file_path, date in zip(file_paths, dates):
#         ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
#         date = pd.to_datetime(date)
#         ds = ds.assign_coords({'tiempo': date})
#         datasets.append(ds)
#     ds_concat=xr.concat(datasets, dim='tiempo')
#     longitudes = ds_concat['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
#     latitudes = ds_concat['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
#     if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north']:
#         ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north': latitudes})
#     else:
#         raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
#     ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east':'longitude'})
#     ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
#     ds_renamed=ds_renamed['TT'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
#     ds_renamed=ds_renamed.mean(['longitude','latitude'])
#     metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
#     return metsDCTime, metsDCValues


# plt.figure
# valuesTVmets,timeTVmets= getdataTVmets()
# plt.scatter(valuesTVmets,timeTVmets)
# valuesMETSPROC,timeMETSPROC= getMetsProc()
# plt.scatter(timeMETSPROC,valuesMETSPROC)
    

## SERIE DE TIEMPO

## ERA5 (Reanálisis Climático de la Era 5)


- Resolución Espacial: 0.25 grados (~27 km)
- Resolución Temporal: 6 Horas
- Periodo 81-2010 en el cluster en la ruta:
<span style="color:red">- /data/datos/ERA/Clim/data</span>


## CFS (Sistema de Pronóstico Climático)


- Resolución Espacial: 0.5 grados (~55 km)
- Resolución Temporal: Mensual
- Período de Pronóstico: Hasta varios meses en el futuro , se esta tomando 7 leads a partir del mes 

## DATOS CFS CORREGIDOS


- Resolución Espacial: 0.25 grados (~27 km)
- Resolución Temporal interpolada: 6 Horas

In [ ]:
### temperatura
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = { 
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}



def getTayloyDiagram(observaciones,predicciones):
    import matplotlib.pyplot as plt
    import numpy as np
    # Calcular el error
    error = observaciones - predicciones
    # Calcular R^2
    r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))
    # Calcular RMS (raíz del error cuadrático medio)
    rms = np.sqrt(np.mean(error ** 2))
    # Calcular S (desviación estándar)
    s = np.std(error)
    # Crear el gráfico de dispersión
    plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')
    # Configurar etiquetas y título
    plt.xlabel('Observaciones')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de Taylor')
    # Agregar línea de referencia 1:1
    plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')
    # Mostrar leyenda
    plt.legend()
    plt.savefig(f'mudaValues.jpg')
    add_image_with_title_to_presentation(f'mudaValues.jpg', ppt, 'Taylor-Digrama')
    # Mostrar el gráfico
    plt.show()








def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
    import xarray as xr
    import pandas as pd
    import numpy as np
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    #ds_concat.to_netcdf('/home/mandrade/Python-mejorado/ncCompilado.nc')   # -> genera los mets compilados en uno solo
    longitudes = ds_concat['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
    if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north']:
        ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['TT'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    metsDCValues=ds_renamed.values-273.15
    metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
    return metsDCTime, metsDCValues
                #0, 10, 80, 90
def graficaTemp(la,la2,lon,lon2,title):
    varCrudla=-la2
    varCrudla2=-la
    varCrudlon=-lon2
    varCrudlon2=-lon
    la, la2, lon, lon2 = -la, -la2, lon2, lon
    latitud, latitud2, longitud, longitud2 = la, la2, 360 - lon, 360 - lon2
    ##################              Cargar datos FNL
    presion = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-tmp.nc')
    presion_sel = presion['TMPprs'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),lev=100000)
    presion_mean = presion_sel.mean(dim=['lat', 'lon'])
    anio_objetivo = 2021
    fecha_original = presion_mean.time.to_index()
    diferencia_anios = anio_objetivo - fecha_original.year[0]
    fecha_nueva = fecha_original + pd.DateOffset(years=diferencia_anios)
    presion_mean_copia = presion_mean.copy(deep=True)
    presion_mean_copia['time'] = fecha_nueva + pd.DateOffset(years=1)
    presion_mean_extendida = xr.concat([presion_mean, presion_mean_copia], dim='time')
    presion_mean_extendida=presion_mean_extendida.sortby('time')
    presion_mean=presion_mean_extendida
    presion_mean['time'] = pd.to_datetime(presion_mean.time) - pd.DateOffset(years=11)
    timeFNL = presion_mean['time']
    valuesFNL = presion_mean.values
    ####################  Cargar datos de flujos
    # flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    # flx = flx.sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2))
    # flx = flx.resample(time='M').mean()
    # flx = flx.mean(dim=['latitud', 'longitud'])
    # valuesFLX = flx['t'].values
    # timeFLX = flx['time']
    # timeFLX=pd.to_datetime(timeFLX)
    # timeFLX=timeFLX - pd.DateOffset(days=29)
    #####################  Cargar datos de Mets
    ds2 = xr.open_dataset('~/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
    longitudes = ds2['XLONG_M']
    latitudes = ds2['XLAT_M']
    ds2 = ds2.assign_coords({'west_east': longitudes.isel(Time=1).values[0, :],'south_north': latitudes.isel(Time=1).values[:, 0]})
    ds2 = ds2.sel(num_metgrid_levels=0, south_north=slice(latitud2, latitud), west_east=slice(longitud - 360, longitud2 - 360))
    ds2 = ds2.mean(dim=['south_north', 'west_east'])
    timeMets = ds2['Time']
    valuesMets = ds2['TT'].values
    ####################### obtener data cruda FIO ###################
    
    metsDCTime,metsDCValues = getdataTVmets(varCrudla,varCrudla2,varCrudlon,varCrudlon2)
    plt.figure(figsize=(16, 6))
    plt.scatter(metsDCTime,metsDCValues,s=100,label='DATA CRUDA')
    # plt.scatter(timeFLX.values, valuesFLX-273.15, label='CLIMATOLOGÍA CFS ', s=85, c='green')
    plt.scatter(timeMets.values, valuesMets-273.15, c='blue', label='CLIMATOLOGÍA - PREPROCESADA',alpha=0.6, s=6)
    plt.scatter(timeFNL.values, valuesFNL-273.15, label='CLIMATOLOGÍA FNL', c='r',s=6)
    plt.ylabel('Temperatura del aire a 2m (°C)')
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%B'))
    fmin = pd.to_datetime('12-01-2011')
    fmax = pd.to_datetime('06-02-2012')
    plt.xlim(fmin, fmax)
    plt.grid(True)
    plt.legend()
    plt.savefig(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg')
    add_image_with_title_to_presentation(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg', ppt, title)
    plt.show()
    return timeFNL,valuesFNL,timeMets,valuesMets
    
timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets']-273.15,df_merged['valuesFNL']-273.15)


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets']-273.15,df_merged['valuesFNL']-273.15)


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets']-273.15,df_merged['valuesFNL']-273.15)


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets']-273.15,df_merged['valuesFNL']-273.15)


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets']-273.15,df_merged['valuesFNL']-273.15)


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets']-273.15,df_merged['valuesFNL']-273.15)



In [ ]:
#### PRESS ###### LISTO
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = { 
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}


def getTayloyDiagram(observaciones,predicciones):
    import matplotlib.pyplot as plt
    import numpy as np
    # Calcular el error
    error = observaciones - predicciones
    # Calcular R^2
    r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))
    # Calcular RMS (raíz del error cuadrático medio)
    rms = np.sqrt(np.mean(error ** 2))
    # Calcular S (desviación estándar)
    s = np.std(error)
    # Crear el gráfico de dispersión
    plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')
    # Configurar etiquetas y título
    plt.xlabel('Observaciones')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de Taylor')
    # Agregar línea de referencia 1:1
    plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')
    # Mostrar leyenda
    plt.legend()
    plt.savefig(f'mudaValues.jpg')
    add_image_with_title_to_presentation(f'mudaValues.jpg', ppt, 'Taylor-Digrama')
    # Mostrar el gráfico
    plt.show()


def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
    import xarray as xr
    import pandas as pd
    import numpy as np
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    longitudes = ds_concat['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
    if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north']:
        ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['PRES'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    metsDCValues=ds_renamed.values
    metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
    return metsDCTime, metsDCValues
                #0, 10, 80, 90
def graficaTemp(la,la2,lon,lon2,title):
    varCrudla=-la2
    varCrudla2=-la
    varCrudlon=-lon2
    varCrudlon2=-lon
    la, la2, lon, lon2 = -la, -la2, lon2, lon
    latitud, latitud2, longitud, longitud2 = la, la2, 360 - lon, 360 - lon2
    ##################              Cargar datos FNL
    presion = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-pre.nc')
    presion_sel = presion['PRESmsl'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2))
    presion_mean = presion_sel.mean(dim=['lat', 'lon'])
    anio_objetivo = 2021
    fecha_original = presion_mean.time.to_index()
    diferencia_anios = anio_objetivo - fecha_original.year[0]
    fecha_nueva = fecha_original + pd.DateOffset(years=diferencia_anios)
    presion_mean_copia = presion_mean.copy(deep=True)
    presion_mean_copia['time'] = fecha_nueva + pd.DateOffset(years=1)
    presion_mean_extendida = xr.concat([presion_mean, presion_mean_copia], dim='time')
    presion_mean_extendida=presion_mean_extendida.sortby('time')
    presion_mean=presion_mean_extendida
    presion_mean['time'] = pd.to_datetime(presion_mean.time) - pd.DateOffset(years=11)
    timeFNL = presion_mean['time']
    valuesFNL = presion_mean.values
    ####################  Cargar datos de flujos
    # flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    # flx = flx.sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2))
    # flx = flx.resample(time='M').mean()
    # flx = flx.mean(dim=['latitud', 'longitud'])
    # valuesFLX = flx['t'].values
    # timeFLX = flx['time']
    # timeFLX=pd.to_datetime(timeFLX)
    # timeFLX=timeFLX - pd.DateOffset(days=29)
    #####################  Cargar datos de Mets
    ds2 = xr.open_dataset('~/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
    longitudes = ds2['XLONG_M']
    latitudes = ds2['XLAT_M']
    ds2 = ds2.assign_coords({'west_east': longitudes.isel(Time=1).values[0, :],'south_north': latitudes.isel(Time=1).values[:, 0]})
    ds2 = ds2.sel(num_metgrid_levels=0, south_north=slice(latitud2, latitud), west_east=slice(longitud - 360, longitud2 - 360))
    ds2 = ds2.mean(dim=['south_north', 'west_east'])
    timeMets = ds2['Time']
    valuesMets = ds2['PRES'].values
    ####################### obtener data cruda FIO ###################
    
    metsDCTime,metsDCValues = getdataTVmets(varCrudla,varCrudla2,varCrudlon,varCrudlon2)
    plt.figure(figsize=(16, 6))
    plt.scatter(metsDCTime,metsDCValues,s=100,label='DATA CRUDA')
    # plt.scatter(timeFLX.values, valuesFLX-273.15, label='CLIMATOLOGÍA CFS ', s=85, c='green')
    plt.scatter(timeMets.values, valuesMets, c='blue', label='CLIMATOLOGÍA - PREPROCESADA',alpha=0.6, s=6)
    plt.scatter(timeFNL.values, valuesFNL-273.15, label='CLIMATOLOGÍA FNL', c='r',s=6)
    plt.ylabel('Presion a 2m (Pa)')
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%B'))
    fmin = pd.to_datetime('12-01-2011')
    fmax = pd.to_datetime('06-02-2012')
    plt.xlim(fmin, fmax)
    plt.grid(True)
    plt.legend()
    plt.savefig(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg')
    add_image_with_title_to_presentation(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg', ppt, title)
    plt.show()
    return timeFNL,valuesFNL,timeMets,valuesMets



timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])



timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])



timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])



timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])



timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])



timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])

In [ ]:
#### RH  ####
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = { 
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}

def getTayloyDiagram(observaciones,predicciones):
    import matplotlib.pyplot as plt
    import numpy as np
    # Calcular el error
    error = observaciones - predicciones
    # Calcular R^2
    r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))
    # Calcular RMS (raíz del error cuadrático medio)
    rms = np.sqrt(np.mean(error ** 2))
    # Calcular S (desviación estándar)
    s = np.std(error)
    # Crear el gráfico de dispersión
    plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')
    # Configurar etiquetas y título
    plt.xlabel('Observaciones')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de Taylor')
    # Agregar línea de referencia 1:1
    plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')
    # Mostrar leyenda
    plt.legend()
    plt.savefig(f'mudaValues.jpg')
    add_image_with_title_to_presentation(f'mudaValues.jpg', ppt, 'Taylor-Digrama')
    # Mostrar el gráfico
    plt.show()

def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
    import xarray as xr
    import pandas as pd
    import numpy as np
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    longitudes = ds_concat['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
    if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north']:
        ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['RH'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    metsDCValues=ds_renamed.values
    metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
    return metsDCTime, metsDCValues
                #0, 10, 80, 90
def graficaTemp(la,la2,lon,lon2,title):
    varCrudla=-la2
    varCrudla2=-la
    varCrudlon=-lon2
    varCrudlon2=-lon
    la, la2, lon, lon2 = -la, -la2, lon2, lon
    latitud, latitud2, longitud, longitud2 = la, la2, 360 - lon, 360 - lon2
    ##################              Cargar datos FNL
    presion = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-rhp.nc')
    presion_sel = presion['RHprs'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),lev_3=100000)
    presion_mean = presion_sel.mean(dim=['lat', 'lon'])
    anio_objetivo = 2021
    fecha_original = presion_mean.time.to_index()
    diferencia_anios = anio_objetivo - fecha_original.year[0]
    fecha_nueva = fecha_original + pd.DateOffset(years=diferencia_anios)
    presion_mean_copia = presion_mean.copy(deep=True)
    presion_mean_copia['time'] = fecha_nueva + pd.DateOffset(years=1)
    presion_mean_extendida = xr.concat([presion_mean, presion_mean_copia], dim='time')
    presion_mean_extendida=presion_mean_extendida.sortby('time')
    presion_mean=presion_mean_extendida
    presion_mean['time'] = pd.to_datetime(presion_mean.time) - pd.DateOffset(years=11)
    timeFNL = presion_mean['time']
    valuesFNL = presion_mean.values
    ####################  Cargar datos de flujos
    # flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    # flx = flx.sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2))
    # flx = flx.resample(time='M').mean()
    # flx = flx.mean(dim=['latitud', 'longitud'])
    # valuesFLX = flx['t'].values
    # timeFLX = flx['time']
    # timeFLX=pd.to_datetime(timeFLX)
    # timeFLX=timeFLX - pd.DateOffset(days=29)
    #####################  Cargar datos de Mets
    ds2 = xr.open_dataset('~/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
    longitudes = ds2['XLONG_M']
    latitudes = ds2['XLAT_M']
    ds2 = ds2.assign_coords({'west_east': longitudes.isel(Time=1).values[0, :],'south_north': latitudes.isel(Time=1).values[:, 0]})
    ds2 = ds2.sel(num_metgrid_levels=0, south_north=slice(latitud2, latitud), west_east=slice(longitud - 360, longitud2 - 360))
    ds2 = ds2.mean(dim=['south_north', 'west_east'])
    timeMets = ds2['Time']
    valuesMets = ds2['RH'].values
    ####################### obtener data cruda FIO ###################
    
    metsDCTime,metsDCValues = getdataTVmets(varCrudla,varCrudla2,varCrudlon,varCrudlon2)
    plt.figure(figsize=(16, 6))
    plt.scatter(metsDCTime,metsDCValues,s=100,label='DATA CRUDA')
    # plt.scatter(timeFLX.values, valuesFLX-273.15, label='CLIMATOLOGÍA CFS ', s=85, c='green')
    plt.scatter(timeMets.values, valuesMets, c='blue', label='CLIMATOLOGÍA - PREPROCESADA',alpha=0.6, s=6)
    plt.scatter(timeFNL.values, valuesFNL, label='CLIMATOLOGÍA FNL', c='r',s=6)
    plt.ylabel('Humedad relativa (%)')
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%B'))
    fmin = pd.to_datetime('12-01-2011')
    fmax = pd.to_datetime('06-02-2012')
    plt.xlim(fmin, fmax)
    plt.grid(True)
    plt.legend()
    plt.savefig(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg')
    add_image_with_title_to_presentation(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg', ppt, title)
    plt.show()
    return timeFNL,valuesFNL,timeMets,valuesMets


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])

In [ ]:
#### HGT
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = { 
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}

def getTayloyDiagram(observaciones,predicciones):
    import matplotlib.pyplot as plt
    import numpy as np
    # Calcular el error
    error = observaciones - predicciones
    # Calcular R^2
    r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))
    # Calcular RMS (raíz del error cuadrático medio)
    rms = np.sqrt(np.mean(error ** 2))
    # Calcular S (desviación estándar)
    s = np.std(error)
    # Crear el gráfico de dispersión
    plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')
    # Configurar etiquetas y título
    plt.xlabel('Observaciones')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de Taylor')
    # Agregar línea de referencia 1:1
    plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')
    # Mostrar leyenda
    plt.legend()
    plt.savefig(f'mudaValues.jpg')
    add_image_with_title_to_presentation(f'mudaValues.jpg', ppt, 'Taylor-Digrama')
    # Mostrar el gráfico
    plt.show()

def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
    import xarray as xr
    import pandas as pd
    import numpy as np
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    longitudes = ds_concat['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
    if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north']:
        ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['GHT'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    metsDCValues=ds_renamed.values
    metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
    return metsDCTime, metsDCValues
                #0, 10, 80, 90
def graficaTemp(la,la2,lon,lon2,title):
    varCrudla=-la2
    varCrudla2=-la
    varCrudlon=-lon2
    varCrudlon2=-lon
    la, la2, lon, lon2 = -la, -la2, lon2, lon
    latitud, latitud2, longitud, longitud2 = la, la2, 360 - lon, 360 - lon2
    ##################              Cargar datos FNL
    presion = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-hgt.nc')
    presion_sel = presion['HGTprs'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),lev=100000)
    presion_mean = presion_sel.mean(dim=['lat', 'lon'])
    anio_objetivo = 2021
    fecha_original = presion_mean.time.to_index()
    diferencia_anios = anio_objetivo - fecha_original.year[0]
    fecha_nueva = fecha_original + pd.DateOffset(years=diferencia_anios)
    presion_mean_copia = presion_mean.copy(deep=True)
    presion_mean_copia['time'] = fecha_nueva + pd.DateOffset(years=1)
    presion_mean_extendida = xr.concat([presion_mean, presion_mean_copia], dim='time')
    presion_mean_extendida=presion_mean_extendida.sortby('time')
    presion_mean=presion_mean_extendida
    presion_mean['time'] = pd.to_datetime(presion_mean.time) - pd.DateOffset(years=11)
    timeFNL = presion_mean['time']
    valuesFNL = presion_mean.values
    ####################  Cargar datos de flujos
    # flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    # flx = flx.sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2))
    # flx = flx.resample(time='M').mean()
    # flx = flx.mean(dim=['latitud', 'longitud'])
    # valuesFLX = flx['t'].values
    # timeFLX = flx['time']
    # timeFLX=pd.to_datetime(timeFLX)
    # timeFLX=timeFLX - pd.DateOffset(days=29)
    #####################  Cargar datos de Mets
    ds2 = xr.open_dataset('~/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
    longitudes = ds2['XLONG_M']
    latitudes = ds2['XLAT_M']
    ds2 = ds2.assign_coords({'west_east': longitudes.isel(Time=1).values[0, :],'south_north': latitudes.isel(Time=1).values[:, 0]})
    ds2 = ds2.sel(num_metgrid_levels=0, south_north=slice(latitud2, latitud), west_east=slice(longitud - 360, longitud2 - 360))
    ds2 = ds2.mean(dim=['south_north', 'west_east'])
    timeMets = ds2['Time']
    valuesMets = ds2['GHT'].values
    ####################### obtener data cruda FIO ###################
    
    metsDCTime,metsDCValues = getdataTVmets(varCrudla,varCrudla2,varCrudlon,varCrudlon2)
    plt.figure(figsize=(16, 6))
    plt.scatter(metsDCTime,metsDCValues,s=100,label='DATA CRUDA')
    # plt.scatter(timeFLX.values, valuesFLX-273.15, label='CLIMATOLOGÍA CFS ', s=85, c='green')
    plt.scatter(timeMets.values, valuesMets, c='blue', label='CLIMATOLOGÍA - PREPROCESADA',alpha=0.6, s=6)
    plt.scatter(timeFNL.values, valuesFNL, label='CLIMATOLOGÍA FNL', c='r',s=6)
    plt.ylabel('Altura geopotencial (m)')
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%B'))
    fmin = pd.to_datetime('12-01-2011')
    fmax = pd.to_datetime('06-02-2012')
    plt.xlim(fmin, fmax)
    plt.grid(True)
    plt.legend()
    plt.savefig(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg')
    add_image_with_title_to_presentation(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg', ppt, title)
    plt.show()
    return timeFNL,valuesFNL,timeMets,valuesMets


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])

In [ ]:
#### COMPONENTE U ####
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = { 
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}

def getTayloyDiagram(observaciones,predicciones):
    import matplotlib.pyplot as plt
    import numpy as np
    # Calcular el error
    error = observaciones - predicciones
    # Calcular R^2
    r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))
    # Calcular RMS (raíz del error cuadrático medio)
    rms = np.sqrt(np.mean(error ** 2))
    # Calcular S (desviación estándar)
    s = np.std(error)
    # Crear el gráfico de dispersión
    plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')
    # Configurar etiquetas y título
    plt.xlabel('Observaciones')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de Taylor')
    # Agregar línea de referencia 1:1
    plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')
    # Mostrar leyenda
    plt.legend()
    plt.savefig(f'mudaValues.jpg')
    add_image_with_title_to_presentation(f'mudaValues.jpg', ppt, 'Taylor-Digrama')
    # Mostrar el gráfico
    plt.show()

def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
    import xarray as xr
    import pandas as pd
    import numpy as np
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    longitudes = ds_concat['XLONG_U'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_U'].isel(tiempo=0, Time=0).values[:, 0]
    if longitudes.shape[0] == ds_concat.dims['west_east_stag'] and latitudes.shape[0] == ds_concat.dims['south_north']:
        ds_concat = ds_concat.assign_coords({'west_east_stag': longitudes, 'south_north': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east_stag':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['UU'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    metsDCValues=ds_renamed.values 
    metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
    return metsDCTime, metsDCValues
                #0, 10, 80, 90
def graficaTemp(la,la2,lon,lon2,title):
    varCrudla=-la2
    varCrudla2=-la
    varCrudlon=-lon2
    varCrudlon2=-lon
    la, la2, lon, lon2 = -la, -la2, lon2, lon
    latitud, latitud2, longitud, longitud2 = la, la2, 360 - lon, 360 - lon2
    ### DATOS ERA5
    def obtenerClimatologia(anho=2, inicio=2014, fin=2015,latitud=latitud,latitud2=latitud2,longitud=longitud,longitud2=longitud2):
        latitud = slice(latitud,latitud2)
        longitud = slice(longitud,longitud2)
        era581 = xr.open_mfdataset('/data/datos/ERA/Clim/data/era_*.nc')
        era581 = era581.sel(latitude=latitud, longitude=longitud)
        era581 = era581.mean(dim=['latitude', 'longitude'])
        # Calcular los promedios mensuales para todos los años
        sstValues = era581['u10'].values
        sstTimes = era581.time
        sst_dataarray = xr.DataArray(sstValues, coords={'time': sstTimes}, dims=['time'])
        # Calcular la climatología mensual
        climatologia_mensual = sst_dataarray.groupby('time.month').mean(dim='time')
        # Seleccionar los valores de la climatología mensual para el año 2014
        climatologia_2014 = climatologia_mensual.sel(month=slice(1, 12)).values
        # Replicar la climatología mensual para el año 2014
        num_años = anho  # Un año
        climatologia_rep_2014 = np.tile(climatologia_2014, num_años)
        # Crear un rango de fechas para el año 2014
        fechas_2014 = pd.date_range(start=f'{inicio}-01-01', end=f'{fin}-12-31', freq='M')
        # Crear un DataFrame con las fechas y los valores de la climatología
        data_2014 = pd.DataFrame({'Fecha': fechas_2014, 'Valor_Climatologia': climatologia_rep_2014})
        return data_2014

    df = obtenerClimatologia(2, 2011, 2012,latitud,latitud2,longitud,longitud2)
    ##################              Cargar datos FNL
    presion = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-ugr.nc')
    presion_sel = presion['UGRDprs'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),lev=100000)
    presion_mean = presion_sel.mean(dim=['lat', 'lon'])
    anio_objetivo = 2021
    fecha_original = presion_mean.time.to_index()
    diferencia_anios = anio_objetivo - fecha_original.year[0]
    fecha_nueva = fecha_original + pd.DateOffset(years=diferencia_anios)
    presion_mean_copia = presion_mean.copy(deep=True)
    presion_mean_copia['time'] = fecha_nueva + pd.DateOffset(years=1)
    presion_mean_extendida = xr.concat([presion_mean, presion_mean_copia], dim='time')
    presion_mean_extendida=presion_mean_extendida.sortby('time')
    presion_mean=presion_mean_extendida
    presion_mean['time'] = pd.to_datetime(presion_mean.time) - pd.DateOffset(years=11)
    timeFNL = presion_mean['time']
    valuesFNL = presion_mean.values
    ####################  Cargar datos de flujos
    # flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    # flx = flx.sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2))
    # flx = flx.resample(time='M').mean()
    # flx = flx.mean(dim=['latitud', 'longitud'])
    # valuesFLX = flx['t'].values
    # timeFLX = flx['time']
    # timeFLX=pd.to_datetime(timeFLX)
    # timeFLX=timeFLX - pd.DateOffset(days=29)
    #####################  Cargar datos de Mets
    ds2 = xr.open_dataset('~/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
    longitudes = ds2['XLONG_U']
    latitudes = ds2['XLAT_U']
    ds2 = ds2.assign_coords({'west_east_stag': longitudes.isel(Time=1).values[0, :],'south_north': latitudes.isel(Time=1).values[:, 0]})
    ds2 = ds2.sel(num_metgrid_levels=0, south_north=slice(latitud2, latitud), west_east_stag=slice(longitud - 360, longitud2 - 360))
    ds2 = ds2.mean(dim=['south_north', 'west_east_stag'])
    timeMets = ds2['Time']
    valuesMets = ds2['UU'].values
    ####################### obtener data cruda FIO ###################
    
    metsDCTime,metsDCValues = getdataTVmets(varCrudla,varCrudla2,varCrudlon,varCrudlon2)
    plt.figure(figsize=(16, 6))
    plt.scatter(metsDCTime,metsDCValues,s=100,label='DATA CRUDA')
    plt.scatter(df['Fecha'], df['Valor_Climatologia'] , label='era5',s=120)
    # plt.scatter(timeFLX.values, valuesFLX-273.15, label='CLIMATOLOGÍA CFS ', s=85, c='green')
    plt.scatter(timeMets.values, valuesMets, c='blue', label='CLIMATOLOGÍA - PREPROCESADA',alpha=0.6, s=6)
    plt.scatter(timeFNL.values, valuesFNL, label='CLIMATOLOGÍA FNL', c='r',s=6)
    plt.ylabel('Componente zonal U (m/s))')
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%B'))
    fmin = pd.to_datetime('12-01-2011')
    fmax = pd.to_datetime('06-02-2012')
    plt.xlim(fmin, fmax)
    plt.grid(True)
    plt.legend()
    plt.savefig(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg')
    add_image_with_title_to_presentation(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg', ppt, title)
    plt.show()
    return timeFNL,valuesFNL,timeMets,valuesMets

timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


# graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
# graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
# graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
# graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
# graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
# graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3

In [ ]:
#### VV ####
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates   
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore", category=UserWarning, message="The .xlabels_top attribute is deprecated.")
warnings.filterwarnings("ignore", category=UserWarning, message="The .ylabels_right attribute is deprecated.")
descripciones_var = { 
        'TT': 'Temperatura superficial del aire a 2m (°C)',
        'PRES': 'Presión a nivel del mar (hPa)',
        'RH': 'Humedad Relativa (%)',
        'GHT': 'Altura Geopotencial (m)',
        'UU': 'Componente zonal U del viento a 10m ',
        'VV': 'Componente meridional V del viento a 10m '
    }
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}
meses_dict = {
    '12': 'Diciembre',
    '01': 'Enero',
    '02': 'Febrero',
    '03': 'Marzo',
    '04': 'Abril',
    '05': 'Mayo',
    '06': 'Junio'
}

def getTayloyDiagram(observaciones,predicciones):
    import matplotlib.pyplot as plt
    import numpy as np
    # Calcular el error
    error = observaciones - predicciones
    # Calcular R^2
    r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))
    # Calcular RMS (raíz del error cuadrático medio)
    rms = np.sqrt(np.mean(error ** 2))
    # Calcular S (desviación estándar)
    s = np.std(error)
    # Crear el gráfico de dispersión
    plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')
    # Configurar etiquetas y título
    plt.xlabel('Observaciones')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de Taylor')
    # Agregar línea de referencia 1:1
    plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')
    # Mostrar leyenda
    plt.legend()
    plt.savefig(f'mudaValues.jpg')
    add_image_with_title_to_presentation(f'mudaValues.jpg', ppt, 'Taylor-Digrama')
    # Mostrar el gráfico
    plt.show()

def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
    import xarray as xr
    import pandas as pd
    import numpy as np
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    longitudes = ds_concat['XLONG_V'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_V'].isel(tiempo=0, Time=0).values[:, 0]
    if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north_stag']:
        ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north_stag': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north_stag': 'latitude','west_east':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['VV'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    metsDCValues=ds_renamed.values 
    metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
    return metsDCTime, metsDCValues
                #0, 10, 80, 90
def graficaTemp(la,la2,lon,lon2,title):
    varCrudla=-la2
    varCrudla2=-la
    varCrudlon=-lon2
    varCrudlon2=-lon
    la, la2, lon, lon2 = -la, -la2, lon2, lon
    latitud, latitud2, longitud, longitud2 = la, la2, 360 - lon, 360 - lon2
    ########
    ### DATOS ERA5
    def obtenerClimatologia(anho=2, inicio=2014, fin=2015,latitud=latitud,latitud2=latitud2,longitud=longitud,longitud2=longitud2):
        latitud = slice(latitud,latitud2)
        longitud = slice(longitud,longitud2)
        era581 = xr.open_mfdataset('/data/datos/ERA/Clim/data/era_*.nc')
        era581 = era581.sel(latitude=latitud, longitude=longitud)
        era581 = era581.mean(dim=['latitude', 'longitude'])
        # Calcular los promedios mensuales para todos los años
        sstValues = era581['v10'].values
        sstTimes = era581.time
        sst_dataarray = xr.DataArray(sstValues, coords={'time': sstTimes}, dims=['time'])
        # Calcular la climatología mensual
        climatologia_mensual = sst_dataarray.groupby('time.month').mean(dim='time')
        # Seleccionar los valores de la climatología mensual para el año 2014
        climatologia_2014 = climatologia_mensual.sel(month=slice(1, 12)).values
        # Replicar la climatología mensual para el año 2014
        num_años = anho  # Un año
        climatologia_rep_2014 = np.tile(climatologia_2014, num_años)
        # Crear un rango de fechas para el año 2014
        fechas_2014 = pd.date_range(start=f'{inicio}-01-01', end=f'{fin}-12-31', freq='M')
        # Crear un DataFrame con las fechas y los valores de la climatología
        data_2014 = pd.DataFrame({'Fecha': fechas_2014, 'Valor_Climatologia': climatologia_rep_2014})
        return data_2014

    df = obtenerClimatologia(2, 2011, 2012,latitud,latitud2,longitud,longitud2)

    ##################              Cargar datos FNL
    presion = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-vgr.nc')
    presion_sel = presion['VGRDprs'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),lev=100000)
    presion_mean = presion_sel.mean(dim=['lat', 'lon'])
    anio_objetivo = 2021
    fecha_original = presion_mean.time.to_index()
    diferencia_anios = anio_objetivo - fecha_original.year[0]
    fecha_nueva = fecha_original + pd.DateOffset(years=diferencia_anios)
    presion_mean_copia = presion_mean.copy(deep=True)
    presion_mean_copia['time'] = fecha_nueva + pd.DateOffset(years=1)
    presion_mean_extendida = xr.concat([presion_mean, presion_mean_copia], dim='time')
    presion_mean_extendida=presion_mean_extendida.sortby('time')
    presion_mean=presion_mean_extendida
    presion_mean['time'] = pd.to_datetime(presion_mean.time) - pd.DateOffset(years=11)
    timeFNL = presion_mean['time']
    valuesFNL = presion_mean.values
    ####################  Cargar datos de flujos
    # flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    # flx = flx.sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2))
    # flx = flx.resample(time='M').mean()
    # flx = flx.mean(dim=['latitud', 'longitud'])
    # valuesFLX = flx['t'].values
    # timeFLX = flx['time']
    # timeFLX=pd.to_datetime(timeFLX)
    # timeFLX=timeFLX - pd.DateOffset(days=29)
    #####################  Cargar datos de Mets
    ds2 = xr.open_dataset('~/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
    longitudes = ds2['XLONG_V']
    latitudes = ds2['XLAT_V']
    ds2 = ds2.assign_coords({'west_east': longitudes.isel(Time=1).values[0, :],'south_north_stag': latitudes.isel(Time=1).values[:, 0]})
    ds2 = ds2.sel(num_metgrid_levels=0, south_north_stag=slice(latitud2, latitud), west_east=slice(longitud - 360, longitud2 - 360))
    ds2 = ds2.mean(dim=['south_north_stag', 'west_east'])
    timeMets = ds2['Time']
    valuesMets = ds2['VV'].values
    ####################### obtener data cruda FIO ###################
    
    metsDCTime,metsDCValues = getdataTVmets(varCrudla,varCrudla2,varCrudlon,varCrudlon2)
    plt.figure(figsize=(16, 6))
    plt.scatter(df['Fecha'], df['Valor_Climatologia'] , label='era5',s=120)
    plt.scatter(metsDCTime,metsDCValues,s=100,label='DATA CRUDA')
    # plt.scatter(timeFLX.values, valuesFLX-273.15, label='CLIMATOLOGÍA CFS ', s=85, c='green')
    plt.scatter(timeMets.values, valuesMets, c='blue', label='CLIMATOLOGÍA - PREPROCESADA',alpha=0.6, s=6)
    plt.scatter(timeFNL.values, valuesFNL, label='CLIMATOLOGÍA FNL', c='r',s=6)
    plt.ylabel('Componente zonal V (m/s))')
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%B'))
    fmin = pd.to_datetime('12-01-2011')
    fmax = pd.to_datetime('06-02-2012')
    plt.xlim(fmin, fmax)
    plt.grid(True)
    plt.legend()
    plt.savefig(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg')
    add_image_with_title_to_presentation(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg', ppt, title)
    plt.show()
    return timeFNL,valuesFNL,timeMets,valuesMets
# graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
# graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
# graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
# graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
# graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
# graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3

timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(0, 10, 80, 90,'REGION 1+2: 0-10S 80-90W') # region 1+2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5, 7, 82.5, 85,'ZONA O1: 3.5-7S 82.5-85W') # O1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11, 13.5, 78, 80,'ZONA O2: 11-13.5S 78-80W') # O2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(4, 6, 79.3, 81,'ZONA C1: 4-6S 79.3-81W') # C1
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(3.5,5.5,72,75,'ZONA C2: 3.5-5.5S 72-75W') #C2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])


timeFNL,valuesFNL,timeMets,valuesMets= graficaTemp(11,13.5,74,76,'ZONA C3: 11-13.5S 74-76W') #C3
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])

# ppt.save('diapositivas.pptx')

In [ ]:
import xarray as xr
MC=xr.open_dataset('/home/mandrade/Python-mejorado/ncCompilado.nc')
l1=-10
l2=0
w1=-90
w2=-80
longitudes = MC['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
latitudes = MC['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
if longitudes.shape[0] == MC.dims['west_east'] and latitudes.shape[0] == MC.dims['south_north']:
        MC = MC.assign_coords({'west_east': longitudes, 'south_north': latitudes})
else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
MC = MC.rename({'south_north': 'latitude','west_east':'longitude'})
MC=MC.sel(Time=0,num_metgrid_levels=0)
MC=MC['TT'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2),tiempo=MC.tiempo[0])
MC.plot()
# MC=MC.mean(['longitude','latitude'])
# metsDCValues=MC.values
# metsDCTime=pd.to_datetime(MC.tiempo) #+pd.DateOffset(years=2)



In [ ]:
1

# HACER VARIACION DIURNA

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
import cartopy.crs as ccrs
import cartopy
import warnings
import locale
import matplotlib.pyplot as plt
def getdataTVmets(l1=-10,l2=0,w1=-90,w2=-80):
    # prueba
    file_paths = [    '/home/mandrade/Data/met_em.d01.2014-12-01_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-02_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-03_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-04_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-05_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-06_00:00:00.nc',
        '/home/mandrade/Data/met_em.d01.2014-12-07_00:00:00.nc']
    dates = ['2011-12-01 00:00', '2012-01-01 00:00','2012-02-01 00:00','2012-03-01 00:00','2012-04-01 00:00','2012-05-01 00:00','2012-06-01 00:00']
    datasets = []
    for file_path, date in zip(file_paths, dates):
        ds = xr.open_dataset(file_path)[['TT', 'PRES', 'GHT', 'RH', 'UU', 'VV','XLONG_M','XLAT_M','XLONG_U','XLAT_U','XLAT_V','XLONG_V']]
        date = pd.to_datetime(date)
        ds = ds.assign_coords({'tiempo': date})
        datasets.append(ds)
    ds_concat=xr.concat(datasets, dim='tiempo')
    #ds_concat.to_netcdf('/home/mandrade/Python-mejorado/ncCompilado.nc')   # -> genera los mets compilados en uno solo
    longitudes = ds_concat['XLONG_M'].isel(tiempo=0, Time=0).values[0, :]
    latitudes = ds_concat['XLAT_M'].isel(tiempo=0, Time=0).values[:, 0]
    if longitudes.shape[0] == ds_concat.dims['west_east'] and latitudes.shape[0] == ds_concat.dims['south_north']:
        ds_concat = ds_concat.assign_coords({'west_east': longitudes, 'south_north': latitudes})
    else:
        raise ValueError("Las dimensiones de las coordenadas no coinciden con las dimensiones existentes.")
    ds_renamed = ds_concat.rename({'south_north': 'latitude','west_east':'longitude'})
    ds_renamed =  ds_renamed.sel(Time=0,num_metgrid_levels=0)
    ds_renamed=ds_renamed['TT'].sel(latitude=slice(l1,l2),longitude=slice(w1,w2))
    ds_renamed=ds_renamed.mean(['longitude','latitude'])
    csv=ds_renamed.to_dataframe()
    metsDCValues=ds_renamed.values-273.15
    metsDCTime=pd.to_datetime(ds_renamed.tiempo) #+pd.DateOffset(years=2)
    df=pd.DataFrame()
    df['time']=metsDCTime
    df['TT']=metsDCValues
    df.to_csv('TT_r1m2.csv',index=False)
    return metsDCTime, metsDCValues


def graficaTemp(la=0,la2=10,lon=80,lon2=90,title='r1+2'):
    varCrudla=-la2
    varCrudla2=-la
    varCrudlon=-lon2
    varCrudlon2=-lon
    la, la2, lon, lon2 = -la, -la2, lon2, lon
    latitud, latitud2, longitud, longitud2 = la, la2, 360 - lon, 360 - lon2
    ##################              Cargar datos FNL
    presion = xr.open_dataset('/home/mandrade/Data/BerlinSegura/out2-std2-hourly-mean-vanho-2-tmp.nc')
    presion_sel = presion['TMPprs'].sel(lat=slice(latitud, latitud2), lon=slice(longitud, longitud2),lev=100000)
    presion_mean = presion_sel.mean(dim=['lat', 'lon'])
    anio_objetivo = 2021
    fecha_original = presion_mean.time.to_index()
    diferencia_anios = anio_objetivo - fecha_original.year[0]
    fecha_nueva = fecha_original + pd.DateOffset(years=diferencia_anios)
    presion_mean_copia = presion_mean.copy(deep=True)
    presion_mean_copia['time'] = fecha_nueva + pd.DateOffset(years=1)
    presion_mean_extendida = xr.concat([presion_mean, presion_mean_copia], dim='time')
    presion_mean_extendida=presion_mean_extendida.sortby('time')
    presion_mean=presion_mean_extendida
    presion_mean['time'] = pd.to_datetime(presion_mean.time) - pd.DateOffset(years=11)
    df=pd.DataFrame()
    df['time']=presion_mean['time']
    df['TT']=presion_mean.values
    df.to_csv('fnl_r1m2.csv',index=False)
    timeFNL = presion_mean['time']
    valuesFNL = presion_mean.values
    
    ####################  Cargar datos de flujos
    # flx = xr.open_dataset('~/Python-mejorado/final_flxf_6H_cubic_compilado_met.nc')
    # flx = flx.sel(latitud=slice(latitud, latitud2), longitud=slice(longitud, longitud2))
    # flx = flx.resample(time='M').mean()
    # flx = flx.mean(dim=['latitud', 'longitud'])
    # valuesFLX = flx['t'].values
    # timeFLX = flx['time']
    # timeFLX=pd.to_datetime(timeFLX)
    # timeFLX=timeFLX - pd.DateOffset(days=29)
    #####################  Cargar datos de Mets
    ds2 = xr.open_dataset('~/Python-mejorado/coords_6H_linear_compilado_met_2.nc')
    longitudes = ds2['XLONG_M']
    latitudes = ds2['XLAT_M']
    ds2 = ds2.assign_coords({'west_east': longitudes.isel(Time=1).values[0, :],'south_north': latitudes.isel(Time=1).values[:, 0]})
    ds2 = ds2.sel(num_metgrid_levels=0, south_north=slice(latitud2, latitud), west_east=slice(longitud - 360, longitud2 - 360))
    ds2 = ds2.mean(dim=['south_north', 'west_east'])
    timeMets = ds2['Time']
    valuesMets = ds2['TT'].values
    df=pd.DataFrame()
    df['time']=ds2['Time']
    df['TT']=ds2['TT'].values
    df.to_csv('METS_r1m2.csv',index=False)
    ####################### obtener data cruda FIO ###################
    
    metsDCTime,metsDCValues = getdataTVmets(varCrudla,varCrudla2,varCrudlon,varCrudlon2)
    plt.figure(figsize=(16, 6))
    plt.scatter(metsDCTime,metsDCValues,s=100,label='DATA CRUDA')
    # plt.scatter(timeFLX.values, valuesFLX-273.15, label='CLIMATOLOGÍA CFS ', s=85, c='green')
    plt.scatter(timeMets.values, valuesMets-273.15, c='blue', label='CLIMATOLOGÍA - PREPROCESADA',alpha=0.6, s=6)
    plt.scatter(timeFNL.values, valuesFNL-273.15, label='CLIMATOLOGÍA FNL', c='r',s=6)
    plt.ylabel('Temperatura del aire a 2m (°C)')
    plt.grid(True)
    plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%B'))
    fmin = pd.to_datetime('12-01-2011')
    fmax = pd.to_datetime('06-02-2012')
    plt.xlim(fmin, fmax)
    plt.grid(True)
    plt.legend()
    plt.savefig(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg')
    add_image_with_title_to_presentation(f'Coordenadas{la}{la2}S-{lon}{lon2}W.jpg', ppt, title)
    plt.show()
    return timeFNL,valuesFNL,timeMets,valuesMets

timeFNL,valuesFNL,timeMets,valuesMets=graficaTemp()

In [ ]:
def getTayloyDiagram(observaciones,predicciones):
    import matplotlib.pyplot as plt
    import numpy as np
    # Calcular el error
    error = observaciones - predicciones
    # Calcular R^2
    r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))
    # Calcular RMS (raíz del error cuadrático medio)
    rms = np.sqrt(np.mean(error ** 2))
    # Calcular S (desviación estándar)
    s = np.std(error)
    # Crear el gráfico de dispersión
    plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')
    # Configurar etiquetas y título
    plt.xlabel('Observaciones')
    plt.ylabel('Predicciones')
    plt.title('Diagrama de Taylor')
    # Agregar línea de referencia 1:1
    plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')
    # Mostrar leyenda
    plt.legend()
    # Mostrar el gráfico
    plt.show()

import pandas as pd

# Crear DataFrames df1 y df2
df1 = pd.DataFrame({'timeFNL': timeFNL, 'valuesFNL': valuesFNL})
df2 = pd.DataFrame({'timeMets': timeMets, 'valuesMets': valuesMets})

# Fusionar los DataFrames y retener solo las filas que existen en ambos
df_merged = pd.merge(df1, df2, how='inner', left_on='timeFNL', right_on='timeMets')

# Ahora df_merged contiene solo las filas que existen en timeFNL y timeMets
getTayloyDiagram(df_merged['valuesMets'],df_merged['valuesFNL'])

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import pandas as pd

# Load the provided CSV files
fnl_r1m2_path = 'fnl_r1m2.csv'
mets_r1m2_path = 'METS_r1m2.csv'
tt_r1m2_path = 'TT_r1m2.csv'

fnl_r1m2_df = pd.read_csv(fnl_r1m2_path)
mets_r1m2_df = pd.read_csv(mets_r1m2_path)
tt_r1m2_df = pd.read_csv(tt_r1m2_path)

# Display the first few rows of each dataframe for inspection
fnl_r1m2_df.head(), mets_r1m2_df.head(), tt_r1m2_df.head()
# Importing necessary libraries for datetime operations
from datetime import datetime

# Parsing the 'time' columns in each dataframe to datetime format
# Assuming the format is 'YYYY-MM-DD HH:MM:SS'
fnl_r1m2_df['time'] = pd.to_datetime(fnl_r1m2_df['time'])
mets_r1m2_df['time'] = pd.to_datetime(mets_r1m2_df['time'])
tt_r1m2_df['time'] = pd.to_datetime(tt_r1m2_df['time'])

# Creating a function to extract the day and month
def extract_day_month(datetime_series):
    return datetime_series.dt.strftime('%m-%d')

# Applying the function to each dataframe
fnl_r1m2_df['day_month'] = extract_day_month(fnl_r1m2_df['time'])
mets_r1m2_df['day_month'] = extract_day_month(mets_r1m2_df['time'])
tt_r1m2_df['day_month'] = extract_day_month(tt_r1m2_df['time'])

# Grouping the fnl_r1m2_df by day_month to get average values for each day_month
fnl_r1m2_grouped = fnl_r1m2_df.groupby('day_month').mean().reset_index()

# Displaying the grouped data
fnl_r1m2_grouped.head()
# Merging the grouped fnl_r1m2 data with the mets_r1m2 data on day_month
merged_df = pd.merge(mets_r1m2_df, fnl_r1m2_grouped, on='day_month', suffixes=('_mets', '_fnl'))

# Calculating the difference between the TT values from fnl_r1m2 and a reference value
# Assuming the reference is the mean of TT from TT_r1m2
reference_value = tt_r1m2_df['TT'].mean()
merged_df['difference'] = merged_df['TT_fnl'] - reference_value

# Applying this difference to the METS_r1m2 data to adjust for the diurnal variation
merged_df['TT_mets_adjusted'] = merged_df['TT_mets'] + merged_df['difference']

# Displaying the first few rows of the merged and adjusted dataframe
merged_df.head()

netcdf_path = '~/Python-mejorado/metsJuntos.nc'

netcdf_data = xr.open_dataset(netcdf_path)

# Suponiendo que ya tienes los datos de fnl_r1m2 en un DataFrame de Pandas llamado fnl_r1m2_df
# Calcula la media horaria de los datos de fnl_r1m2
fnl_r1m2_df['hour'] = pd.to_datetime(fnl_r1m2_df['time']).dt.hour
hourly_avg = fnl_r1m2_df.groupby('hour')['TT'].mean()

# Extrae la variable TT
tt_data = netcdf_data['TT']

# Selecciona un subconjunto de datos
subset_tt_data = tt_data.isel(south_north=slice(0, 10), west_east=slice(0, 10))

# Interpola cada 6 horas
subset_interpolated = subset_tt_data.resample(tiempo='6H').interpolate('linear')

# Aplica la variación cíclica
hourly_variation = np.array([hourly_avg.get(hour, 0) for hour in [0, 6, 12, 18]])
time_repeats = subset_interpolated.sizes['tiempo'] // 4
hourly_variation_repeated = np.tile(hourly_variation, time_repeats)
subset_interpolated_adjusted = subset_interpolated + (hourly_variation_repeated - hourly_variation_repeated.mean())

# Guarda el subconjunto ajustado en un nuevo archivo NetCDF si es necesario
# subset_interpolated_adjusted.to_netcdf('/ruta/a/tu/nuevo/archivo.nc')



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Datos de ejemplo (observaciones y predicciones)
observaciones = np.array([10, 15, 20, 25, 30])
predicciones = np.array([9, 14, 18, 24, 29])

# Calcular el error
error = observaciones - predicciones

# Calcular R^2
r_squared = 1 - (np.sum(error ** 2) / np.sum((observaciones - np.mean(observaciones)) ** 2))

# Calcular RMS (raíz del error cuadrático medio)
rms = np.sqrt(np.mean(error ** 2))

# Calcular S (desviación estándar)
s = np.std(error)

# Crear el gráfico de dispersión
plt.scatter(observaciones, predicciones, c='b', label=f'R^2: {r_squared:.2f}\nRMS: {rms:.2f}\nS: {s:.2f}')

# Configurar etiquetas y título
plt.xlabel('Observaciones')
plt.ylabel('Predicciones')
plt.title('Diagrama de Taylor')

# Agregar línea de referencia 1:1
plt.plot([min(observaciones), max(observaciones)], [min(observaciones), max(observaciones)], 'k--', label='1:1')

# Mostrar leyenda
plt.legend()

# Mostrar el gráfico
plt.show()
